# ELephantSQL Database Builder:

A script for importing local aviation data; modifying it to create various tables of interest; and then importing those tables into a database so that my web app can access it

By Kenneth Burchfiel

Released under the MIT License

In [1]:
import json
import sqlalchemy
import pandas as pd
import time

Connecting to ElephantSQL database:

In [2]:
with open ("..\key_paths\path_to_keys_folder.txt") as file:
    key_path = file.read()

In [3]:
with open(key_path+"\\elephantsql_kjb3webchartapp_db_url.txt") as file:
    db_url = file.read()
    # This code reads in my database's URL. This URL is listed on the home page for my database within elephantsql.com. As shown below, SQLAlchemy can use this URL to connect to the database. 

elephantsql_db_url_for_sqlalchemy = db_url.replace('postgres://', 'postgresql://')
# This change, which is required for SQLAlchemy to work correctly, is based on the code suggested at:
# # https://help.heroku.com/ZKNTJQSK/why-is-sqlalchemy-1-4-x-not-connecting-to-heroku-postgres

In [4]:
elephantsql_engine = sqlalchemy.create_engine(elephantsql_db_url_for_sqlalchemy)

## Loading in data

Next, I'll load in a .csv file that was created using the [flights_table_builder](https://github.com/kburchfiel/route_maps_builder/blob/master/flights_table_builder.ipynb) script within my [Route Maps Builder](https://github.com/kburchfiel/route_maps_builder) program. The original file is too large to host on GitHub, but it can be found within [this Google Drive folder](https://drive.google.com/drive/folders/1jRTjoZtT6OWCXRTNstG9D4CxmjXe4W8q?usp=sharing) as 'routes_planes_coordinates.csv'.



In [5]:
pd.set_option('display.max_columns', 100)

In [6]:
def df_size_in_megabytes(df):
    return sum(df.memory_usage())/(1024**2) # See https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.memory_usage.html . Dividing it by 1024**2 converts the result from bytes to megabytes.

Reading in airline data:

In [7]:
df_flights = pd.read_csv(r'C:\Users\kburc\D1V1\Documents\!Dell64docs\Programming\py\kjb3_programs\route_maps_builder\routes_planes_coordinates.csv')
df_flights.drop('Unnamed: 0', axis = 1,inplace=True)
df_flights.fillna("Unknown", inplace = True) # This line will allow rows missing a value for a variable to still be included in pivot tables that process that variable.
df_flights

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.64,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.98,-95.34,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,11612,1161206,31612,EVV,"Evansville, IN",IN,18.0,Indiana,US,United States,42,6,629,1,2018,1,2,1,F,DU,Unknown,629,Canadair RJ-200ER /RJ-440,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,Unknown,Unknown,Unknown,Domestic,Domestic,Domestic,EVV_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482268,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,3,7,1,F,DU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.06,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
482269,1188.0,594.0,1069200.0,5346.0,3573.0,0.0,0.0,91.0,27799.0,21740.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,107

In [8]:
len(df_flights['DEST'].unique())

1891

In [9]:
len(df_flights['DEST'].unique())

1891

In [10]:
# Approximate memory usage (in megabytes) of the DataFrame:

df_size_in_megabytes(df_flights)


246.5233383178711

## Pivoting and filtering data

This table currently contains around 350,000 rows and 65 columns. However, my web app doesn't need all of those rows present in its own PostgreSQL database; instead, it just needs enough data to generate various charts. Therefore, I'll now create some modified versions of this data and then upload it to the PostgreSQL database.

In [11]:
df_airline_origin_dest_plane_pivot = df_flights.pivot_table(index = ['UNIQUE_CARRIER', 'ORIGIN_DEST', 'Plane_Type_Text'], values = 'PASSENGERS', aggfunc = 'sum').reset_index().sort_values('PASSENGERS', ascending = False)
df_airline_origin_dest_plane_pivot = df_airline_origin_dest_plane_pivot.query("PASSENGERS > 100000").copy()
df_airline_origin_dest_plane_pivot['Airport 1'] = df_airline_origin_dest_plane_pivot['ORIGIN_DEST'].str.split('_').str[0]
df_airline_origin_dest_plane_pivot['Airport 2'] = df_airline_origin_dest_plane_pivot['ORIGIN_DEST'].str.split('_').str[1]
df_airline_origin_dest_plane_pivot.rename(columns={"UNIQUE_CARRIER":"Airline","PASSENGERS":"Passengers","ORIGIN_DEST":"Origin_Dest"},inplace=True)
df_airline_origin_dest_plane_pivot.reset_index(drop=True,inplace=True)
df_airline_origin_dest_plane_pivot

,Airline,Origin_Dest,Plane_Type_Text,Passengers,Airport 1,Airport 2
0,BA,JFK_LHR,Boeing 747-400,1980000.0,JFK,LHR
1,HA,HNL_OGG,Boeing 717-200,1954139.0,HNL,OGG
2,DL,ATL_MCO,Boeing 757-200,1417832.0,ATL,MCO
3,HA,HNL_KOA,Boeing 717-200,1281221.0,HNL,KOA
4,WN,DAL_HOU,Boeing 737-700/700LR/Max 7,1260362.0,DAL,HOU
...,...,...,...,...,...,...
2991,EI,DUB_MCO,Airbus Industrie A330-200,100329.0,DUB,MCO
2992,YX,DCA_MCI,Embraer ERJ-175,100303.0,DCA,MCI
2993,WN,BNA_CLT,Boeing 737-700/700LR/Max 7,100282.0,BNA,CLT
2994,OO,MSP_RAP,Canadair RJ-200ER /RJ-440,100097.0,MSP,RAP


In [12]:
sum(df_airline_origin_dest_plane_pivot.memory_usage())/(1024**2)

0.13726806640625

The following cell exports this pivot table to .csv format locally (so that it can be accessed without a database connection) and to .sql format online.

In [13]:
df_airline_origin_dest_plane_pivot.to_csv('local_copy_of_airports_airlines_aircraft_2018.csv', index = False)

df_airline_origin_dest_plane_pivot.to_sql("airports_airlines_aircraft_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

996

I believe the output of the above cell shows the smallest 3 digits of the number of rows updated within the table. Thus, if 2,804 rows were created within the table, the output will be 804.

The following cell reads the new database table back into Pandas in order to verify that it was imported successfully:

In [14]:
pd.read_sql("airports_airlines_aircraft_2018", con = elephantsql_engine)

,Airline,Origin_Dest,Plane_Type_Text,Passengers,Airport 1,Airport 2
0,BA,JFK_LHR,Boeing 747-400,1980000.0,JFK,LHR
1,HA,HNL_OGG,Boeing 717-200,1954139.0,HNL,OGG
2,DL,ATL_MCO,Boeing 757-200,1417832.0,ATL,MCO
3,HA,HNL_KOA,Boeing 717-200,1281221.0,HNL,KOA
4,WN,DAL_HOU,Boeing 737-700/700LR/Max 7,1260362.0,DAL,HOU
...,...,...,...,...,...,...
2991,EI,DUB_MCO,Airbus Industrie A330-200,100329.0,DUB,MCO
2992,YX,DCA_MCI,Embraer ERJ-175,100303.0,DCA,MCI
2993,WN,BNA_CLT,Boeing 737-700/700LR/Max 7,100282.0,BNA,CLT
2994,OO,MSP_RAP,Canadair RJ-200ER /RJ-440,100097.0,MSP,RAP


## Creating additional tables for the database:

The following cells create filtered versions of df_flights that show only domestic or international destinations. This allows me to ensure, for instance, that a list of the top 20 US airports by international traffic won't contain non-US airports.

In [15]:
df_flights_domestic_destination_airports = df_flights.query("Destination_Region == 'Domestic'")
df_flights_domestic_destination_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.64,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.98,-95.34,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,11612,1161206,31612,EVV,"Evansville, IN",IN,18.0,Indiana,US,United States,42,6,629,1,2018,1,2,1,F,DU,Unknown,629,Canadair RJ-200ER /RJ-440,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,Unknown,Unknown,Unknown,Domestic,Domestic,Domestic,EVV_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482268,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,3,7,1,F,DU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.06,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
482269,1188.0,594.0,1069200.0,5346.0,3573.0,0.0,0.0,91.0,27799.0,21740.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,107

In [16]:
df_flights_intl_destination_airports = df_flights.query("Destination_Region == 'International'")
df_flights_intl_destination_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
44,0.0,1.0,21502.0,76.0,66.0,0.0,0.0,182.0,43.0,32.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,14576,1457606,34576,ROC,"Rochester, NY",NY,36.0,New York,US,United States,22,16149,1614904,36149,YOW,"Ottawa, Canada",ON,Unknown,Ontario,CA,Canada,936,6,638,1,2018,1,2,1,F,IU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ROC,43.119,-77.672,YOW,45.323,-75.669,Domestic,International,International,ROC_YOW
146,0.0,1.0,21405.0,0.0,0.0,0.0,0.0,1335.0,232.0,220.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,11292,1129202,30325,DEN,"Denver, CO",CO,8.0,Colorado,US,United States,82,11874,1187406,31874,GDL,"Guadalajara, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,6,635,2,2018,1,2,3,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,DEN,39.858,-104.667,GDL,20.522,-103.311,Domestic,International,International,DEN_GDL
147,0.0,1.0,21148.0,0.0,0.0,6840.0,0.0,1348.0,218.0,197.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,11612,1161206,31612,EVV,"Evansville, IN",IN,18.0,Indiana,US,United States,42,14879,1487901,34879,SLP,"San Luis Potosi, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,6,635,2,2018,1,2,3,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,Unknown,Unknown,Unknown,SLP,22.254,-100.931,Domestic,International,International,EVV_SLP
148,0.0,1.0,20995.0,0.0,0.0,10740.0,0.0,1052.0,198.0,170.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,12217,1221702,30255,HSV,"Huntsville, AL",AL,1.0,Alabama,US,United States,51,14886,1488605,34886,SLW,"Saltillo, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,6,635,2,2018,1,2,3,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,Unknown,Unknown,Unknown,SLW,25.549,-100.929,Domestic,International,International,HSV_SLW
149,0.0,1.0,21323.0,0.0,0.0,5090.0,0.0,165.0,47.0,33.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,14886,1488605,34886,SLW,"Saltillo, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,6,635,2,2018,1,2,1,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,LRD,27.544,-99.461,SLW,25.549,-100.929,Domestic,International,International,LRD_SLW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480682,246.0,123.0,221400.0,1107.0,455.0,0.0,0.0,96.0,5904.0,4797.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,14843,1484306,34819,SJU,"San Juan, PR",PR,72.0,Puerto Rico,US,United States,3,11519,1151907,34448,EIS,"Road Town, British Virgin Islands",Unknown,Unknown,Unknown,VG,British Virgin Islands,282,1,125,1,2018,3,8,1,F,IU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,SJU,18.439,-66.002,EIS,18.445,-64.543,Domestic,Inte

In [17]:
df_flights_domestic_destination_airports_intl_traffic = df_flights.query("Destination_Region == 'Domestic' & Route_Type == 'International'")
df_flights_domestic_destination_airports_intl_traffic

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
52,0.0,1.0,21502.0,76.0,75.0,0.0,0.0,803.0,256.0,203.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,16217,1621702,36083,YUL,"Montreal, Canada",QC,Unknown,Quebec,CA,Canada,941,11057,1105703,31057,CLT,"Charlotte, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,2,F,IU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,YUL,45.468,-73.741,CLT,35.214,-80.943,International,Domestic,International,CLT_YUL
145,0.0,1.0,21323.0,0.0,0.0,0.0,0.0,468.0,75.0,60.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,10638,1063802,30638,BJX,"Leon/Guanajuato, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,BJX,20.993,-101.481,LRD,27.544,-99.461,International,Domestic,International,BJX_LRD
152,0.0,2.0,42400.0,0.0,0.0,0.0,0.0,128.0,82.0,54.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,13514,1351403,33514,MTY,"Monterrey, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,MTY,25.778,-100.107,LRD,27.544,-99.461,International,Domestic,International,LRD_MTY
156,0.0,1.0,21148.0,0.0,0.0,0.0,0.0,376.0,80.0,60.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,14879,1487901,34879,SLP,"San Luis Potosi, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,SLP,22.254,-100.931,LRD,27.544,-99.461,International,Domestic,International,LRD_SLP
157,0.0,3.0,63313.0,0.0,0.0,0.0,0.0,165.0,141.0,87.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,14886,1488605,34886,SLW,"Saltillo, Mexico",Unknown,Unknown,Unknown,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,Unknown,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,SLW,25.549,-100.929,LRD,27.544,-99.461,International,Domestic,International,LRD_SLW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480738,248.0,124.0,223200.0,1116.0,663.0,0.0,0.0,96.0,5952.0,4836.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,11519,1151907,34448,EIS,"Road Town, British Virgin Islands",Unknown,Unknown,Unknown,VG,British Virgin Islands,282,14843,1484306,34819,SJU,"San Juan, PR",PR,72.0,Puerto Rico,US,United States,3,1,125,1,2018,3,8,1,F,IU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,EIS,18.445,-64.543,SJU,18.439,-66.002,International,Domestic,International,EIS_

# Creating a U.S. airports table

I'll now create a list of the top 20 U.S. airports by arriving passenger traffic that will also include (1) domestic and international traffic breakdowns and (2) each airport's rank. (I chose to focus on arriving traffic rather than departing traffic, but either option can work, as long as you're consistent in that decision throughout your code.) First, I'll calculate the ranks of the top 20 U.S. airports by arriving passenger traffic:

In [18]:
df_top_20_airport_ranks = df_flights_domestic_destination_airports.pivot_table(index = "DEST", values = "PASSENGERS", aggfunc = "sum").sort_values("PASSENGERS", ascending = False).head(20).reset_index()
df_top_20_airport_ranks['Rank'] = df_top_20_airport_ranks['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airport_ranks

,DEST,PASSENGERS,Rank
0,ATL,52811864.0,1
1,LAX,44366710.0,2
2,ORD,40804852.0,3
3,DFW,33597743.0,4
4,JFK,32650440.0,5
5,DEN,32148783.0,6
6,SFO,28976834.0,7
7,LAS,24392133.0,8
8,SEA,24355042.0,9
9,MCO,23538779.0,10


Next, I'll create a list of these top 20 airports:

In [19]:
top_20_airports = df_top_20_airport_ranks['DEST'].to_list()
top_20_airports

['ATL',
 'LAX',
 'ORD',
 'DFW',
 'JFK',
 'DEN',
 'SFO',
 'LAS',
 'SEA',
 'MCO',
 'EWR',
 'CLT',
 'PHX',
 'MIA',
 'IAH',
 'BOS',
 'MSP',
 'FLL',
 'DTW',
 'PHL']

I'll then use this list to create a filtered version of df_flights:

In [20]:
df_top_20_domestic_airports = df_flights.query("DEST in @top_20_airports")
df_top_20_domestic_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.64,-73.779,Domestic,Domestic,Domestic,IAD_JFK
13,0.0,1.0,21502.0,76.0,36.0,0.0,0.0,304.0,93.0,65.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12945,1294503,32945,LEX,"Lexington, KY",KY,21.0,Kentucky,US,United States,52,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,Unknown,Unknown,Unknown,ATL,33.64,-84.427,Domestic,Domestic,Domestic,ATL_LEX
14,0.0,2.0,43004.0,152.0,114.0,0.0,0.0,762.0,322.0,264.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12953,1295304,31703,LGA,"New York, NY",NY,36.0,New York,US,United States,22,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,LGA,40.777,-73.872,ATL,33.64,-84.427,Domestic,Domestic,Domestic,ATL_LGA
19,0.0,1.0,21502.0,76.0,67.0,0.0,0.0,304.0,80.0,57.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12992,1299206,32600,LIT,"Little Rock, AR",AR,5.0,Arkansas,US,United States,71,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,LIT,34.729,-92.224,DFW,32.896,-97.037,Domestic,Domestic,Domestic,DFW_LIT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482259,910.0,455.0,819000.0,4095.0,2360.0,0.0,0.0,91.0,21294.0,16653.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,2,5,1,F,DU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.06,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
482264,1096.0,548.0,986400.0,4932.0,2913.0,0.0,0.0,91.0,25646.0,20057.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,

In [21]:
df_top_20_domestic_airports_pivot = df_top_20_domestic_airports.pivot_table(index = ['DEST', 'Route_Type'], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).reset_index()
df_top_20_domestic_airports_pivot = df_top_20_domestic_airports_pivot.merge(df_top_20_airport_ranks[['DEST', 'Rank']], on = 'DEST')
df_top_20_domestic_airports_pivot.sort_values(['Rank', 'Route_Type'], inplace = True)
df_top_20_domestic_airports_pivot.rename(columns={"DEST":"Airport","PASSENGERS":"Passengers"}, inplace = True)
df_top_20_domestic_airports_pivot

,Airport,Route_Type,Passengers,Rank
0,ATL,Domestic,46292899.0,1
1,ATL,International,6518965.0,1
6,LAX,Domestic,30439363.0,2
7,LAX,International,13927347.0,2
2,ORD,Domestic,33415706.0,3
3,ORD,International,7389146.0,3
8,DFW,Domestic,29020269.0,4
9,DFW,International,4577474.0,4
23,JFK,Domestic,14279281.0,5
22,JFK,International,18371159.0,5


I'm also interested in seeing which U.S. airports had the highest international passenger traffic, so I'll create that list as well:

In [22]:
df_top_20_airports_intl_traffic = df_flights_domestic_destination_airports_intl_traffic.pivot_table(index = 'destination_iata_code', values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).head(20) # Calculates the top 20 airports by arriving passenger count.
df_top_20_airports_intl_traffic.reset_index(inplace=True)
df_top_20_airports_intl_traffic['Rank'] = df_top_20_airports_intl_traffic['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airports_intl_traffic.rename(columns={"destination_iata_code":"Airport","PASSENGERS":"Passengers"},inplace=True)
df_top_20_airports_intl_traffic

,Airport,Passengers,Rank
0,JFK,18371159.0,1
1,LAX,13927347.0,2
2,MIA,11413572.0,3
3,SFO,7785713.0,4
4,ORD,7389146.0,5
5,EWR,7384671.0,6
6,ATL,6518965.0,7
7,IAH,5545207.0,8
8,DFW,4577474.0,9
9,FLL,4398074.0,10


Now that I've created these tables, I'll go ahead and export them to both a local .csv file and to my SQL database:

In [23]:
df_size_in_megabytes(df_top_20_airports_intl_traffic)

0.0005035400390625

In [24]:
df_top_20_airports_intl_traffic.to_csv('top_20_airports_by_intl_pax_arrivals_2018.csv', index = False)

df_top_20_airports_intl_traffic.to_sql("top_20_airports_by_intl_pax_arrivals_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

20

In [25]:
df_size_in_megabytes(df_top_20_domestic_airports_pivot)

0.001373291015625

In [26]:
df_top_20_domestic_airports_pivot.to_csv('top_20_airports_by_pax_arrivals_2018.csv', index = False)

df_top_20_domestic_airports_pivot.to_sql("top_20_airports_by_pax_arrivals_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

40

# Creating an airlines table

I'll now go through a similar process to create a list of the top 20 airlines by US airport arrivals that distinguishes between domestic and international traffic.

In [27]:
df_top_20_airline_ranks = df_flights.pivot_table(index = 'UNIQUE_CARRIER', values = 'PASSENGERS', aggfunc = 'sum').reset_index().sort_values('PASSENGERS', ascending = False).head(20).reset_index(drop=True)
df_top_20_airline_ranks['Rank'] = df_top_20_airline_ranks['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airline_ranks

,UNIQUE_CARRIER,PASSENGERS,Rank
0,WN,170696369.0,1
1,DL,154006981.0,2
2,AA,152789699.0,3
3,UA,116261088.0,4
4,B6,42608993.0,5
5,OO,39377030.0,6
6,AS,34144232.0,7
7,NK,29273380.0,8
8,F9,19932261.0,9
9,YX,18741699.0,10


In [28]:
df_top_20_airline_list = df_top_20_airline_ranks['UNIQUE_CARRIER'].to_list()
df_top_20_airline_list

['WN',
 'DL',
 'AA',
 'UA',
 'B6',
 'OO',
 'AS',
 'NK',
 'F9',
 'YX',
 'YV',
 'BA',
 'G4',
 '9E',
 'MQ',
 'OH',
 'HA',
 'EV',
 'QX',
 'CP']

In [29]:
df_top_20_airlines = df_flights.query("UNIQUE_CARRIER in @df_top_20_airline_list")
df_top_20_airlines

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.64,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.98,-95.34,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,11612,1161206,31612,EVV,"Evansville, IN",IN,18.0,Indiana,US,United States,42,6,629,1,2018,1,2,1,F,DU,Unknown,629,Canadair RJ-200ER /RJ-440,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,Unknown,Unknown,Unknown,Domestic,Domestic,Domestic,EVV_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482250,855.0,854.0,27027900.0,109312.0,81674.0,172551.0,0.0,100.0,33015.0,19785.0,HA,19690.0,Hawaiian Airlines Inc.,05375,D,HA,Hawaiian Airlines Inc.,3,3.0,13830,1383002,33830,OGG,"Kahului, HI",HI,15.0,Hawaii,US,United States,2,12173,1217305,32134,HNL,"Honolulu, HI",HI,15.0,Hawaii,US,United States,2,6,608,1,2018,4,10,1,F,DU,Unknown,608,Boeing 717-200,6,"Jet, 2-Engine",1,Passenger Configuration,OGG,20.899,-156.431,HNL,21.316,-157.927,Domestic,Domestic,Domestic,HNL_OGG
482252,857.0,859.0,27193800.0,109952.0,81504.0,479954.0,0.0,100.0,36143.0,19001.0,HA,19690.0,Hawaiian Airlin

In [30]:
df_top_20_airlines_pivot = df_top_20_airlines.pivot_table(index = ['UNIQUE_CARRIER', 'Route_Type'], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).reset_index()
df_top_20_airlines_pivot = df_top_20_airlines_pivot.merge(df_top_20_airline_ranks[['UNIQUE_CARRIER', 'Rank']], on = 'UNIQUE_CARRIER')
df_top_20_airlines_pivot.sort_values(['Rank', 'Route_Type'], inplace = True)
df_top_20_airlines_pivot.rename(columns={"UNIQUE_CARRIER":"Airline","PASSENGERS":"Passengers"}, inplace = True)
df_top_20_airlines_pivot

,Airline,Route_Type,Passengers,Rank
0,WN,Domestic,166054984.0,1
1,WN,International,4641385.0,1
2,DL,Domestic,127486190.0,2
3,DL,International,26520791.0,2
4,AA,Domestic,120509970.0,3
5,AA,International,32279729.0,3
6,UA,Domestic,86631450.0,4
7,UA,International,29629638.0,4
10,B6,Domestic,34070233.0,5
11,B6,International,8538760.0,5


In [31]:
df_top_20_airlines_pivot.to_csv('top_20_airlines_by_passengers_2018.csv', index = False)
df_top_20_airlines_pivot.to_sql("top_20_airlines_by_passengers_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

40

# Creating a pivot table showing arriving passensger totals for each airline-airport pair:

In [32]:
df_flights

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.64,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.98,-95.34,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,11612,1161206,31612,EVV,"Evansville, IN",IN,18.0,Indiana,US,United States,42,6,629,1,2018,1,2,1,F,DU,Unknown,629,Canadair RJ-200ER /RJ-440,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,Unknown,Unknown,Unknown,Domestic,Domestic,Domestic,EVV_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482268,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,3,7,1,F,DU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.06,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
482269,1188.0,594.0,1069200.0,5346.0,3573.0,0.0,0.0,91.0,27799.0,21740.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,107

In [33]:
df_airport_airline_pivot = df_flights.pivot_table(index = ['UNIQUE_CARRIER', 'DEST', "Route_Type", "Destination_Region"], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).query("PASSENGERS > 0").reset_index()
df_airport_airline_pivot.rename(columns={"UNIQUE_CARRIER":"Airline","DEST":"Airport","PASSENGERS":"Passengers"},inplace=True)
df_airport_airline_pivot

,Airline,Airport,Route_Type,Destination_Region,Passengers
0,DL,ATL,Domestic,Domestic,33475381.0
1,AA,DFW,Domestic,Domestic,19787311.0
2,AA,CLT,Domestic,Domestic,12531987.0
3,UA,ORD,Domestic,Domestic,10611338.0
4,WN,MDW,Domestic,Domestic,10212889.0
...,...,...,...,...,...
11623,AN,RIC,Domestic,Domestic,1.0
11624,1HQ,HPN,Domestic,Domestic,1.0
11625,15Q,KJK,International,International,1.0
11626,VJT,PIE,International,Domestic,1.0


The output of the following two cells should match; otherwise, I've lost some passengers in the process of creating my pivot table.

In [34]:
df_airport_airline_pivot['Passengers'].sum()

1051331376.0

In [35]:
df_flights['PASSENGERS'].sum()

1051331376.0

In [36]:
df_size_in_megabytes(df_airport_airline_pivot)

0.443695068359375

In [37]:
df_airport_airline_pivot.to_csv('airport_airline_pairs_2018.csv', index = False)
df_airport_airline_pivot.to_sql('airline_airport_pairs_2018', if_exists = 'replace', con = elephantsql_engine, index = False)

628

# Creating a sample of the data that can be used for an interactive comparison chart:

In order to demonstrate how Plotly can be used to create a chart that lets users choose on which variables to compare data, I'll create a simplified version of the table that contains all of the top 4 airlines' passenger traffic at all of the top 5 airports. This table will also distinguish between domestic and international traffic.

In [38]:
top_5_airports = df_flights_domestic_destination_airports.pivot_table(index = "DEST", values = "PASSENGERS", aggfunc = "sum").sort_values("PASSENGERS", ascending = False).index[0:5].to_list()
top_5_airports

['ATL', 'LAX', 'ORD', 'DFW', 'JFK']

In [39]:
top_4_airlines = df_flights.pivot_table(index = "UNIQUE_CARRIER", values = "PASSENGERS", aggfunc = "sum").sort_values("PASSENGERS", ascending = False).index[0:4].to_list()
top_4_airlines

['WN', 'DL', 'AA', 'UA']

In [40]:
df_flights_excerpt_for_pivot_table = df_flights_domestic_destination_airports.query("UNIQUE_CARRIER in @top_4_airlines & DEST in @top_5_airports")
df_flights_excerpt_for_pivot_table

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
5875,0.0,1.0,103900.0,273.0,186.0,0.0,0.0,731.0,137.0,112.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,627,1,2018,4,11,2,L,DU,Unknown,627,Boeing 777-200ER/200LR/233LR,6,"Jet, 2-Engine",1,Passenger Configuration,ATL,33.64,-84.427,DFW,32.896,-97.037,Domestic,Domestic,Domestic,ATL_DFW
5876,0.0,1.0,50200.0,181.0,134.0,0.0,0.0,731.0,142.0,117.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,699,1,2018,4,11,2,L,DU,Unknown,699,Airbus Industrie A321/Lr,6,"Jet, 2-Engine",1,Passenger Configuration,ATL,33.64,-84.427,DFW,32.896,-97.037,Domestic,Domestic,Domestic,ATL_DFW
5878,0.0,1.0,50200.0,181.0,175.0,0.0,0.0,1037.0,176.0,160.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,11027,1102705,31027,CKB,"Clarksburg/Fairmont, WV",WV,54.0,West Virginia,US,United States,39,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,699,1,2018,4,11,3,L,DU,Unknown,699,Airbus Industrie A321/Lr,6,"Jet, 2-Engine",1,Passenger Configuration,Unknown,Unknown,Unknown,DFW,32.896,-97.037,Domestic,Domestic,Domestic,CKB_DFW
5883,0.0,1.0,103900.0,273.0,176.0,0.0,0.0,731.0,107.0,90.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,627,1,2018,4,11,2,L,DU,Unknown,627,Boeing 777-200ER/200LR/233LR,6,"Jet, 2-Engine",1,Passenger Configuration,DFW,32.896,-97.037,ATL,33.64,-84.427,Domestic,Domestic,Domestic,ATL_DFW
5884,0.0,1.0,50200.0,181.0,135.0,0.0,0.0,731.0,120.0,99.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,699,1,2018,4,11,2,L,DU,Unknown,699,Airbus Industrie A321/Lr,6,"Jet, 2-Engine",1,Passenger Configuration,DFW,32.896,-97.037,ATL,33.64,-84.427,Domestic,Domestic,Domestic,ATL_DFW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482013,412.0,411.0,20461760.0,81783.0,75597.0,93143.0,8766.0,404.0,37671.0,25620.0,DL,19790.0,Delta Air Lines Inc.,01260,D,DL,Delta Air Lines Inc.,3,3.0,13204,1320402,31454,MCO,"Orlando, FL",FL,12.0,Florida,US,United States,33,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,622,1,2018,3,8,1,F,DU,Unknown,622,Boeing 757-200,6,"Jet, 2-Engine",1,Passenger Configuration,MCO,28.429,-81.316,ATL,33.64,-84.427,Domestic,Domest

The following pivot table contains rows for each airline/airport/route type pair (e.g. ATL-DL-International; DFW-AA-Domestic, etc.). Since there are 5 airports, 4 airlines, and 2 route types, the pivot table could have up to 40 rows. In practice, however, it will contain a smaller number, since not all of these airlines fly domestically and internationally from every airport.

In [41]:
df_t5_airports_t4_airlines_pivot = df_flights_excerpt_for_pivot_table.pivot_table(index = ["DEST", "UNIQUE_CARRIER", "Route_Type"], values = "PASSENGERS", aggfunc = "sum").reset_index().sort_values(['DEST', 'UNIQUE_CARRIER', 'Route_Type']).reset_index(drop=True)
df_t5_airports_t4_airlines_pivot.rename(columns={"DEST":"Airport","UNIQUE_CARRIER":"Airline","PASSENGERS":"Passengers"},inplace=True)
df_t5_airports_t4_airlines_pivot

,Airport,Airline,Route_Type,Passengers
0,ATL,AA,Domestic,1271957.0
1,ATL,AA,International,399.0
2,ATL,DL,Domestic,33475381.0
3,ATL,DL,International,5061938.0
4,ATL,UA,Domestic,566861.0
5,ATL,WN,Domestic,5116127.0
6,ATL,WN,International,89823.0
7,DFW,AA,Domestic,19787311.0
8,DFW,AA,International,3047763.0
9,DFW,DL,Domestic,816340.0


In [42]:
df_t5_airports_t4_airlines_pivot.to_csv('t5_airports_t4_airlines_2018.csv', index = False)
df_t5_airports_t4_airlines_pivot.to_sql("t5_airports_t4_airlines_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

35

In [43]:
df_flights.columns

Index(['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'PAYLOAD', 'SEATS',
       'PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'RAMP_TO_RAMP', 'AIR_TIME',
       'UNIQUE_CARRIER', 'AIRLINE_ID', 'UNIQUE_CARRIER_NAME',
       'UNIQUE_CARRIER_ENTITY', 'REGION', 'CARRIER', 'CARRIER_NAME',
       'CARRIER_GROUP', 'CARRIER_GROUP_NEW', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'DEST_WAC', 'AIRCRAFT_GROUP', 'AIRCRAFT_TYPE', 'AIRCRAFT_CONFIG',
       'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP', 'CLASS', 'DATA_SOURCE',
       'Unnamed: 50', 'Code_x', 'Plane_Type_Text', 'Code_y',
       'Plane_Grou

In [44]:
df_flights

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.64,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.98,-95.34,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,Unknown,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,11612,1161206,31612,EVV,"Evansville, IN",IN,18.0,Indiana,US,United States,42,6,629,1,2018,1,2,1,F,DU,Unknown,629,Canadair RJ-200ER /RJ-440,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,Unknown,Unknown,Unknown,Domestic,Domestic,Domestic,EVV_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482268,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,3,7,1,F,DU,Unknown,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.06,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
482269,1188.0,594.0,1069200.0,5346.0,3573.0,0.0,0.0,91.0,27799.0,21740.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,107

In [45]:
df_dest_by_origin = df_flights.pivot_table(index = ['DEST', 'ORIGIN'], values = 'PASSENGERS', aggfunc = 'sum').reset_index()
df_dest_by_origin.rename(columns={"DEST":"Airport","ORIGIN":"Origin", "PASSENGERS":"Passengers"}, inplace = True)
df_dest_by_origin

,Airport,Origin,Passengers
0,05A,CXF,5.0
1,05A,FAI,242.0
2,05A,SCC,0.0
3,05A,WCR,7.0
4,06A,09A,0.0
...,...,...,...
35625,ZXU,MVY,2.0
35626,ZXU,PKB,2.0
35627,ZXU,PSF,3.0
35628,ZXU,TEB,7.0


In [46]:
df_size_in_megabytes(df_dest_by_origin)

0.8156280517578125

In [47]:
df_dest_by_origin.to_csv('dest_to_origin_2018.csv', index = False)
df_dest_by_origin.to_sql('dest_to_origin', if_exists = 'replace', index = False,
con = elephantsql_engine)

630

In [48]:
print(time.ctime())

Fri Nov 18 00:15:54 2022
